In [1]:
import numpy as np
import pandas as pd
import glob
import cv2
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
train = pd.read_csv(r'../input/petfinder-pawpularity-score/train.csv')
train

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15
9908,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70
9909,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20
9910,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20


In [3]:
image_list = []
filenames = list(glob.glob('../input/petfinder-pawpularity-score/train/*.jpg'))
files_ids = [x.split('/')[-1].split('.')[-2] for x in filenames]

In [4]:
for filename in tqdm(filenames):
    im=cv2.imread(filename, cv2.IMREAD_COLOR)
    im = cv2.resize(im,(224,224))
    image_list.append(im)
image_list=np.array(image_list)
image_list.shape

100%|██████████| 9912/9912 [02:14<00:00, 73.54it/s]


(9912, 224, 224, 3)

In [5]:
model = models.resnet18()
weights = torch.load('../input/resnet18/resnet18.pth')
model.load_state_dict(weights)
layer = model._modules.get('avgpool')
model.eval()
print('Model Loaded')

Model Loaded


In [6]:
to_tensor = transforms.ToTensor()
def get_vector(img):
    t_img = Variable(to_tensor(img)).unsqueeze(0)
    my_embedding = torch.zeros([1, 512, 1, 1])
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    h = layer.register_forward_hook(copy_data)
    model(t_img)
    h.remove()
    my_embedding=my_embedding.cpu().detach().numpy()
    return my_embedding
vector_list = [get_vector(x) for x in tqdm(image_list)]
vector_list = np.array(vector_list)
vector_list = vector_list[:,0,:,0,0]
vector_df = pd.DataFrame([files_ids]).T
vector_df.columns = ['Id']
vector_df = pd.concat([vector_df, pd.DataFrame(vector_list)], axis=1)
vector_df

100%|██████████| 9912/9912 [08:52<00:00, 18.60it/s]


,Id,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,7954ebb5c90d9618e34959df0ad5f062,0.380061,0.048842,1.442434,1.599896,0.710317,0.381115,1.005558,1.863736,3.550770,...,0.358193,0.304743,0.436533,2.950845,1.659424,0.464157,1.454988,4.203649,0.332297,1.316501
1,2969162fab1d0e5a65e4ce02db267745,2.876565,1.658750,0.751358,1.671523,1.163443,0.187763,0.615759,1.720958,2.153686,...,0.377098,1.302348,0.241997,1.217116,2.073370,0.835313,0.620960,1.618683,0.582233,0.666297
2,9f32ae94d7723414cb8fa881a1c6626c,1.620128,1.139042,0.301076,4.849315,1.707183,0.229410,0.000906,3.075203,0.545536,...,1.243727,0.145624,0.100839,0.473114,1.799587,0.464690,0.813697,3.447953,0.548108,0.604041
3,b8f920f44800ee71e4659dea84bc9bef,0.872216,0.332755,0.798420,3.322190,0.016920,0.442386,0.030731,0.326005,1.551584,...,0.789491,0.837104,0.457777,1.941184,1.074297,0.134871,1.281129,3.645407,0.023303,1.192199
4,7fc71b8da143721939715b1cfe22122f,0.971978,1.281718,1.778319,1.422007,1.057356,0.347828,0.311844,2.120642,1.211101,...,1.255148,0.522153,0.329352,0.493841,0.951819,0.589563,1.188267,1.898910,1.656261,0.132997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,d22da16a635031c1ae7554c839f26c9b,0.307226,0.504647,0.152517,1.200827,1.023759,0.829910,0.468882,1.585457,1.823001,...,1.049986,0.017313,0.309109,1.192471,0.205988,0.105267,0.292615,2.236170,1.324818,0.767677
9908,54ef90ca5972af9ebdb54fb5fcd67552,0.494517,0.262002,0.493978,2.550193,0.378372,0.341678,0.421689,0.710373,0.610251,...,0.637921,0.022332,0.090197,0.833992,1.561602,0.480402,1.035995,1.818522,0.036011,0.655489
9909,c86fe58fcfb959cda493dc6578c1868c,0.068562,0.093018,1.430016,0.598557,0.077190,0.466088,0.559463,0.005997,2.525880,...,1.478614,0.426698,0.189445,1.145149,0.426843,1.042881,1.283682,1.701080,0.158010,0.857879
9910,44293284a7b4ac2a2e4ebfa57db4c48c,1.190700,0.315497,0.382405,3.334412,0.955070,0.299990,0.113048,1.188560,0.802559,...,1.002464,0.799387,0.165273,0.955335,0.709802,1.324229,0.441008,2.417794,0.474491,0.189742


In [7]:
X = pd.merge(train, vector_df, on='Id', how='left')
y = X['Pawpularity'].values
X = X.drop(columns = ['Pawpularity','Id'])
X.columns = X.columns.astype(str)
X.astype(float)

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,...,502,503,504,505,506,507,508,509,510,511
0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.900238,0.007091,0.000000,1.671684,0.435511,0.036351,0.124130,2.216105,1.065397,0.391920
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.422475,1.634731,0.129232,0.802656,1.050760,1.197792,2.118588,5.146822,0.441037,2.025496
2,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.420526,0.569457,0.032572,0.875901,1.254961,0.150740,0.865864,2.789046,0.163807,0.527404
3,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.475000,0.048889,0.014187,1.436925,3.115373,0.043731,1.607734,1.363528,0.148980,0.809543
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.100896,0.115165,0.542225,1.620398,1.335747,0.366337,0.212317,3.380268,0.145514,2.092883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.563875,0.354742,0.034918,0.270625,1.314700,0.844234,1.876923,2.635725,0.350934,0.169699
9908,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.485318,0.448113,0.113586,0.391468,1.209488,0.063895,1.168252,1.636838,0.450858,0.467315
9909,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.110886,0.209386,0.541494,1.631402,1.446667,0.626202,1.983186,3.152981,0.186353,0.688972
9910,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.512441,0.613686,0.119038,0.366918,1.207764,0.346634,0.350562,2.613190,0.584035,0.267461


In [8]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
# X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.90, random_state=0)
ftest = SelectKBest(score_func=f_regression, k='all')
ftest.fit(X,y)
f_scores = pd.DataFrame(ftest.scores_)
dfcolumns = pd.DataFrame(X.columns)
f_scores = pd.concat([dfcolumns,f_scores],axis=1)
f_scores.columns = ['Feature','F-Score']  

In [9]:
f_scores = f_scores.sort_values(by='F-Score',ascending=True)[:26]
f_scores

,Feature,F-Score
89,77,0.000240
502,490,0.000342
233,221,0.000999
439,427,0.001843
53,41,0.002560
303,291,0.002917
160,148,0.003196
516,504,0.003596
344,332,0.003863
47,35,0.009076


In [10]:
# X_train = X_train.drop(f_scores.Feature.tolist(), axis=1)
# X_test=X_test.drop(f_scores.Feature.tolist(), axis=1)
X=X.drop(f_scores.Feature.tolist(), axis=1)

In [11]:
# type(X_train)
X

,Subject Focus,Eyes,Face,Accessory,Group,Human,Info,Blur,0,1,...,501,502,503,505,506,507,508,509,510,511
0,0,1,1,0,1,0,0,0,1.083370,0.390286,...,1.319900,0.900238,0.007091,1.671684,0.435511,0.036351,0.124130,2.216105,1.065397,0.391920
1,0,1,1,0,0,0,0,0,2.220349,0.218677,...,3.454426,0.422475,1.634731,0.802656,1.050760,1.197792,2.118588,5.146822,0.441037,2.025496
2,0,1,1,0,0,1,0,0,0.683794,0.727704,...,2.099775,0.420526,0.569457,0.875901,1.254961,0.150740,0.865864,2.789046,0.163807,0.527404
3,0,1,1,0,0,0,0,0,0.224811,0.452718,...,1.327836,0.475000,0.048889,1.436925,3.115373,0.043731,1.607734,1.363528,0.148980,0.809543
4,0,0,0,0,1,0,0,0,1.267661,0.836573,...,3.813493,0.100896,0.115165,1.620398,1.335747,0.366337,0.212317,3.380268,0.145514,2.092883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,0,0,0,0,0,0,0,1,0.309975,0.170929,...,2.964248,0.563875,0.354742,0.270625,1.314700,0.844234,1.876923,2.635725,0.350934,0.169699
9908,0,1,1,0,0,0,0,0,0.487469,0.659123,...,1.426332,0.485318,0.448113,0.391468,1.209488,0.063895,1.168252,1.636838,0.450858,0.467315
9909,0,1,1,0,0,1,0,0,1.096129,0.806347,...,2.230231,1.110886,0.209386,1.631402,1.446667,0.626202,1.983186,3.152981,0.186353,0.688972
9910,0,1,1,0,0,1,0,0,1.005058,1.670247,...,1.685588,0.512441,0.613686,0.366918,1.207764,0.346634,0.350562,2.613190,0.584035,0.267461


In [12]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
model_1 = SVR(kernel='rbf')
# from sklearn.tree import DecisionTreeRegressor
# model_2 = DecisionTreeRegressor(random_state=0)
# from math import sqrt
from sklearn.ensemble import VotingRegressor
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_test_scaled = scaler.transform(X_test)
# ensemble_model.fit(X_train_scaled,y)
from sklearn.linear_model import Ridge
model_3 = Ridge(alpha=2.0)
ensemble_model = VotingRegressor(estimators=[('svr',model_1),
                                              ('ridge',model_3)]
                                ,weights=[1,1])
ensemble_model.fit(X_scaled,y)
# y_preds = ensemble_model.predict(X_scaled)
# sqrt(mean_squared_error(y_test, y_preds))

VotingRegressor(estimators=[('svr', SVR()), ('ridge', Ridge(alpha=2.0))],
                weights=[1, 1])

In [13]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# model = Sequential()
# model.add(Dense(524, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(1, activation='relu'))

In [14]:
# model.compile(loss='mse',optimizer='Adam',metrics=['accuracy'])
# model.fit(X_train_scaled,y)

In [15]:
test = pd.read_csv(r'../input/petfinder-pawpularity-score/test.csv')
test_image_list = []
test_filenames = list(glob.glob('../input/petfinder-pawpularity-score/test/*.jpg'))
test_files_ids = [x.split('/')[-1].split('.')[-2] for x in test_filenames]
for test_filename in tqdm(test_filenames):
    im=cv2.imread(filename, cv2.IMREAD_COLOR)
    im = cv2.resize(im,(224,224))
    test_image_list.append(im)

100%|██████████| 8/8 [00:00<00:00, 113.32it/s]


In [16]:
test_image_list=np.array(test_image_list)
test_vector_list = [get_vector(x) for x in tqdm(test_image_list)]
test_vector_list = np.array(test_vector_list)
test_vector_list = test_vector_list[:,0,:,0,0]
test_vector_df = pd.DataFrame([test_files_ids]).T
test_vector_df.columns = ['Id']
test_vector_df = pd.concat([test_vector_df, pd.DataFrame(test_vector_list)], axis=1)
test_vector_df

100%|██████████| 8/8 [00:00<00:00, 19.57it/s]


,Id,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,c978013571258ed6d4637f6e8cc9d6a3,0.462609,0.717984,1.808226,1.058193,1.437242,0.722915,0.629291,0.526725,1.052403,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
1,4e429cead1848a298432a0acad014c9d,0.462609,0.717984,1.808226,1.058193,1.437242,0.722915,0.629291,0.526725,1.052403,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
2,43a2262d7738e3d420d453815151079e,0.462609,0.717984,1.808226,1.058193,1.437242,0.722915,0.629291,0.526725,1.052403,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
3,8f49844c382931444e68dffbe20228f4,0.462609,0.717984,1.808226,1.058193,1.437242,0.722915,0.629291,0.526725,1.052403,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
4,4128bae22183829d2b5fea10effdb0c3,0.462609,0.717984,1.808226,1.058193,1.437242,0.722915,0.629291,0.526725,1.052403,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
5,80bc3ccafcc51b66303c2c263aa38486,0.462609,0.717984,1.808226,1.058193,1.437242,0.722915,0.629291,0.526725,1.052403,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
6,e0de453c1bffc20c22b072b34b54e50f,0.462609,0.717984,1.808226,1.058193,1.437242,0.722915,0.629291,0.526725,1.052403,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
7,b03f7041962238a7c9d6537e22f9b017,0.462609,0.717984,1.808226,1.058193,1.437242,0.722915,0.629291,0.526725,1.052403,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515


In [17]:
X_test = pd.merge(test, test_vector_df, on='Id', how='left')
X_test = X_test.drop(columns = ['Id'])
X_test

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,...,502,503,504,505,506,507,508,509,510,511
0,1,0,1,0,0,1,1,0,0,1,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
1,0,1,0,0,0,0,1,1,0,0,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
2,0,0,0,1,0,1,1,1,0,1,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
3,1,0,1,0,0,0,0,0,0,0,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
4,1,1,1,0,1,1,0,1,0,1,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
5,0,0,1,1,1,1,1,1,1,0,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
6,1,0,0,0,1,1,0,1,0,1,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
7,1,0,1,0,0,0,0,0,1,0,...,0.634608,0.594277,0.0,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515


In [18]:
X_test.columns = X_test.columns.astype(str)
X_test_final = X_test.drop(f_scores.Feature.tolist(), axis = 1)
X_test_scaled = scaler.transform(X_test_final)
# X_test = X_test.drop(columns = f_scores.Feature.tolist(), axis=1)
# X_test.columns = X_test.columns.astype(str)
# X_test_scaled = scaler.transform(X_test)
X_test_final

,Subject Focus,Eyes,Face,Accessory,Group,Human,Info,Blur,0,1,...,501,502,503,505,506,507,508,509,510,511
0,1,0,1,1,1,0,0,1,0.462609,0.717984,...,1.949048,0.634608,0.594277,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
1,0,1,0,0,1,0,0,0,0.462609,0.717984,...,1.949048,0.634608,0.594277,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
2,0,0,0,1,1,0,1,1,0.462609,0.717984,...,1.949048,0.634608,0.594277,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
3,1,0,1,0,0,0,1,0,0.462609,0.717984,...,1.949048,0.634608,0.594277,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
4,1,1,1,1,0,0,1,0,0.462609,0.717984,...,1.949048,0.634608,0.594277,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
5,0,0,1,1,1,1,1,0,0.462609,0.717984,...,1.949048,0.634608,0.594277,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
6,1,0,0,1,0,0,1,1,0.462609,0.717984,...,1.949048,0.634608,0.594277,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515
7,1,0,1,0,0,1,0,1,0.462609,0.717984,...,1.949048,0.634608,0.594277,1.835488,1.561908,0.602881,2.442821,2.959185,0.07341,0.084515


In [19]:
y_pred = ensemble_model.predict(X_test_scaled)
y_pred
test['Pawpularity'] = y_pred.tolist()
test_results = test.drop(columns = ['Subject Focus','Eyes','Face','Near', 'Action','Accessory','Group','Collage','Human','Occlusion','Info','Blur'])
test_results

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,33.263065
1,43a2262d7738e3d420d453815151079e,34.835866
2,4e429cead1848a298432a0acad014c9d,32.963569
3,80bc3ccafcc51b66303c2c263aa38486,35.011706
4,8f49844c382931444e68dffbe20228f4,34.639658
5,b03f7041962238a7c9d6537e22f9b017,35.806819
6,c978013571258ed6d4637f6e8cc9d6a3,31.937870
7,e0de453c1bffc20c22b072b34b54e50f,32.191907


In [20]:
test_results.to_csv('/kaggle/working/submission.csv',index=False)

In [21]:
test_results.to_csv('submission.csv',index=False)